# Libraries

In [1]:
import pandas as pd
import numpy as np
import re

#import matplotlib.pyplot as plt
#import seaborn as sns

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.preprocessing import sequence

2024-02-15 14:00:11.936123: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 14:00:11.972202: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-15 14:00:11.972230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-15 14:00:11.972248: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-15 14:00:11.977918: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-15 14:00:11.978470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

# Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
fake_df = pd.read_csv('/content/drive/MyDrive/TFM/Dataset/Fake.csv')
true_df = pd.read_csv('/content/drive/MyDrive/TFM/Dataset/True.csv')

In [ ]:
fake_df.info()

In [ ]:
true_df.info()

In [ ]:
fake_df.head()

In [ ]:
true_df.head()

In [ ]:
fake_df.isnull().sum()

In [ ]:
true_df.isnull().sum()

In [ ]:
fake_df.subject.unique()

In [ ]:
true_df.subject.unique()

In [ ]:
fake_df.drop(['subject', 'date'], axis=1, inplace=True)
true_df.drop(['subject', 'date'], axis=1, inplace=True)

In [ ]:
fake_df['class'] = 0
true_df['class'] = 1

In [ ]:
fake_df.head()

In [ ]:
true_df.head()

In [ ]:
news_df = pd.concat([fake_df, true_df], ignore_index=True, sort=False)

In [ ]:
news_df.head()

In [ ]:
news_df.tail()

In [ ]:
news_df['text'] = news_df['title'] + news_df['text']
news_df.drop(['title'], axis=1, inplace=True)

In [ ]:
news_df.head()

In [ ]:
news_df.iloc[5,0]

# Features Engineering

In [ ]:
!pip install -U sentence-transformers
# https://www.sbert.net/

In [3]:
from sentence_transformers import SentenceTransformer
model_embedding = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
embeddings = model_embedding.encode(news_df['text'])
embeddings

In [ ]:
X = embeddings
y = news_df['class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

# Model Build & Training

In [ ]:
# Input for variable-length sequences of integers
model = Sequential()
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, y_train, batch_size=50, epochs=2, validation_data=(X_test, y_test))

# Cross Validation

In [ ]:
Y_predict = model.predict(X_test)
Y_predict

In [ ]:
#redondea las predicciones a 1 y 0
Y_predict = (Y_predict > 0.5).astype(int)

In [ ]:
#lo transforma todo a una sóla lista
Y_predict = [x[0] for x in Y_predict]
Y_predict

In [ ]:
results_df = pd.DataFrame({
    'Test': y_test,
    'Predicción': Y_predict
})

results_df

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, Y_predict)


In [ ]:
import numpy as np
from sklearn.metrics import f1_score
f1_score(y_test, Y_predict, average='weighted')

# Model Save

In [ ]:
model.save('./drive/MyDrive/TFM/tfmodel')

In [ ]:
# tfmodel = './drive/MyDrive/TFM/tfmodel'
# !zip -r tfmodel.zip tfmodel
!zip -r tfmodel.zip ./drive/MyDrive/TFM/tfmodel

# Model Reload

In [4]:
model_test = load_model('../models/tfmodel')

# Model Test

In [12]:
def news_check(news):
  news_list=[]
  news_list.append(news)
  embeddings_news_list = model_embedding.encode(news_list)
  # embeddings_news_list = model_embedding.encode(news)

  # prediction = model.predict(embeddings_test)
  prediction = model_test.predict(embeddings_news_list)

  if prediction > 0.5:
    return 'La noticia es Verdadera'
  else:
    return 'La noticia es Falsa'


In [16]:
news=input('Ingresa una noticia en ingles y luego presiona "enter": ')
news_check(news)

1/1 [==============================] - 0s 14ms/step


'La noticia es Verdadera'